In [17]:
import cv2
import time
import os
import numpy as np
import cv2
import glob
import ntpath
import os
from PIL import Image
from numpy import asarray


# Function to Extract Frames from Video

In [18]:
def video_to_frames(input_loc, output_loc):
    """Function to extract frames from input video file
    and save them as separate frames in an output directory.
    Args:
        input_loc: Input video file.
        output_loc: Output directory to save the frames.
    Returns:
        None
    """
    try:
        os.mkdir(output_loc)
    except OSError:
        pass
    # Log the time
    time_start = time.time()
    # Start capturing the feed
    cap = cv2.VideoCapture(input_loc)
    # Find the number of frames
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print ("Number of frames: ", video_length)
    count = 0
    print ("Converting video..\n")
    # Start converting the video
    while cap.isOpened():
        # Extract the frame
        ret, frame = cap.read()
        #print(frame.shape)
        #print(frame[0])
        #print(np.amax(frame))
        #break
        # Write the results back to output location.
        cv2.imwrite(output_loc + "/%#05d.jpg" % (count+1), frame)
        count = count + 1
        # If there are no more frames left
        if (count > (video_length-1)):
            # Log the time again
            time_end = time.time()
            # Release the feed
            cap.release()
            # Print stats
            print ("Done extracting frames.\n%d frames extracted" % count)
            print ("It took %d seconds for conversion." % (time_end-time_start))
            break
    return count

# Segmentation

In [19]:
def frameconversion():
    print ("\n Converting frames . . .\n")
    filelist=glob.glob('./frames/*.jpg')
    for i in filelist:
        image = cv2.imread(i)
        #print(type(image))
        #print(image[0])

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        blurred = cv2.GaussianBlur(gray, (11, 11), cv2.BORDER_DEFAULT)
        ret,th1 = cv2.threshold(blurred,170,255,cv2.THRESH_BINARY)


        # gaussian_thresh= cv2.adaptiveThreshold(blurred,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,201,0)

        #cv2.imshow("binary",th1)

        #cv2.waitKey(0)
        #cv2.destroyAllWindows()

        # original = image.copy()
        # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # blurred = cv2.GaussianBlur(gray, (3, 3), 0)
        # #canny = cv2.Canny(blurred, 120, 255, 1)
        # #kernel = np.ones((5,5),np.uint8)
        # #dilate = cv2.dilate(canny, kernel, iterations=1)
        filename=ntpath.basename(i)
        pre=filename.split(".")[0]
        fi='./step2/'+pre+".jpg"
        cv2.imwrite(fi,th1)
    print ("Applied Gaussian Blur and Threshold . . .\n")

In [20]:
def removeflir():
    filelist=glob.glob('./step2/*.jpg')
    for i in filelist:
        image=cv2.imread(i)
        rectangle = np.array([[3,475],[160, 475], [165, 405], [7, 405]])
        color = [0, 0, 0] #black
        cv2.fillConvexPoly(image, rectangle, color)
        filename=ntpath.basename(i)
        pre=filename.split(".")[0]
        fi='./step2/'+pre+".jpg"
        cv2.imwrite(fi,image)
    print ("Removed FLIR watermark . . .\n")
        

In [446]:
##Get Information about White Parts 
def datapreprocessing():
    filelist=glob.glob('./step2/*.jpg')
    result_array = np.array([])

    counter = 0
    for i in filelist:
        if counter == 20:
            counter = 0
            image = cv2.imread(i)
            occurrences = np.count_nonzero(image == 255)
            result_array = np.append(result_array, occurrences)
            #print(occurrences)
        counter += 1
    #print(type(result_array[0]))
    
    ## Normalise The DATA and save the Normalised Data to NPZ File

    startvalue=result_array[0]

    data=np.array([])

    counter = 0
    sample = []
    samples = []
    #extract normalised data
    for i in result_array:
        if counter == 10:
            samples.append(sample)
            sample = []
            counter = 0
        devvalue=float(startvalue-i)/float(startvalue)
    #     print(devvalue)
        normvalue=1-devvalue
        #print(normvalue)
        sample.append(normvalue)
        counter += 1

    samples_np = np.array(samples)
    samples_np.shape
    np.savez_compressed('Data/natural/plastic/p_18', samples=samples)   #name of .npz file 
    print("Created .npz file")
    #     data=np.append(data,normvalue)
    
   


In [453]:
def savedataasnpz():
    ##Load data from .npz files
    
    #Coffee Cup Values

    data=[]
    datalabel=[]
    filelist=glob.glob('./Data/combined/cofeecup data/*.npz')
    for i in filelist:
        loaded = np.load(i)['samples']

        for j in loaded:

            data.append(j)
            datalabel.append(0)    # '0' means coffee Cup

    # #Plastic Bottle Values
    filelist=glob.glob('./Data/combined/plastic data/*.npz')
    for i in filelist:
        loaded = np.load(i)['samples']

        for j in loaded:

            data.append(j)
            datalabel.append(1)   # '1' means Plastic Bottle

    #Cigg Butt Values

    filelist=glob.glob('./Data/combined/cigg data/*.npz')
    for i in filelist:
        loaded = np.load(i)['samples']

        for j in loaded:

            data.append(j)
            datalabel.append(2)     #'2' means Ciggerette Butt



    data = np.array(data)
    datalabel = np.array(datalabel)

    print(data.shape)
    print(datalabel.shape)
    
    ##Save the whole data as one .npz file
    
    np.savez_compressed('sample_training_data_combined', data=data, label=datalabel)
    # np.savez_compressed('DataLabel',datalabel)
    #sample_training_data = np.load('sample_training_data.npz')
    #data = sample_training_data['data']
    #labels = sample_training_data['label']
    #print(data.shape)
    #print(labels.shape)
    print ("Saved The Training Data..\n")

In [454]:
if __name__=="__main__":

    input_loc = 'C:/Users/dar/Desktop/Midas - Multiple Objects/Video.mp4'
    output_loc = 'C:/Users/dar/Desktop/Midas - Multiple Objects/step2'
    framecount=video_to_frames(input_loc, output_loc)
    frameconversion()
    removeflir()
    datapreprocessing()
    savedataasnpz()
    
    


(468, 10)
(468,)
Saved The Training Data..

